In [1]:
import numpy as np
from ngsolve import *
from netgen.occ import *
from netgen.meshing import MeshingParameters
from ngsolve.webgui import Draw

lbd = 1064e-3
c0 = 3e14 # µm/s
freq = c0 / lbd
mu0 = 4 * np.pi * 1e-1
eps0 = 1 / (mu0 * c0**2)
omega = 2 * np.pi * freq
k0 = omega / c0

mu_r_air = 1.0
mu_r_copper = 1.0
sigma_copper = 5.8  # S/µm
eps_r_copper = 1 - 1j * sigma_copper / (omega * eps0)
eps_r_air = 1.0

base_x = 100  # en µm
base_y = 100
base_z = 30
rect_x = 5
rect_y = 100
rect_z = 5
spacing = 20
num_elements = int(base_x / (spacing + rect_x))

pml_thickness = 0
total_x = 150
total_y = 150
total_z = 150

full_domain = Box(Pnt(-total_x / 2, -total_y / 2, -total_z / 2),
                  Pnt(total_x / 2, total_y / 2, total_z / 2))

inner_domain = Box(Pnt(-total_x / 2 + pml_thickness,
                       -total_y / 2 + pml_thickness,
                       -total_z / 2 + pml_thickness),
                   Pnt(total_x / 2 - pml_thickness,
                       total_y / 2 - pml_thickness,
                       total_z / 2 - pml_thickness))

pml_region = full_domain - inner_domain
pml_region.mat("pml")

base = Box(Pnt(-base_x / 2, -base_y / 2, -base_z / 2),
           Pnt(base_x / 2, base_y / 2, base_z / 2))

copper_region = base
for i in range(num_elements):
    current_x = -((num_elements - 1) * spacing) / 2 + i * spacing
    grating_element = Box(Pnt(-rect_x / 2, -rect_y / 2, base_z / 2),
                          Pnt(rect_x / 2, rect_y / 2, base_z / 2 + rect_z)).Move(Vec(current_x, 0, 0))
    copper_region = copper_region + grating_element

copper_region.maxh = 5.0
copper_region.mat("copper")

air_region = inner_domain - copper_region
air_region.mat("air")

# Glue all the regions together
geo = OCCGeometry(Glue([air_region, copper_region, pml_region]))

# --- Mesh Generation ---
# Set global mesh size
mp = MeshingParameters(maxh=15.0)

# Generate the mesh. The local maxh on the copper region will override the global maxh there.
ngmesh = geo.GenerateMesh(mp=mp)
mesh = Mesh(ngmesh)
print("Mesh generated.")

# Draw the mesh to verify the result
Draw(mesh)

Mesh generated.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [2]:
mesh.SetPML(pml.Cartesian(mins = (-total_x/2, -total_y/2, -total_z/2),
                          maxs = (total_x/2, total_y/2, total_z/2),
                          alpha=1j*k0), "pml")

fes = HCurl(mesh, order=2, complex=True)

E_scat = fes.TrialFunction()
v = fes.TestFunction()

E_inc_coeff = CoefficientFunction( (1.0,0, 0) )
E_inc = E_inc_coeff * exp(-1j * k0 * z)
E_inc_curl = CoefficientFunction( (0, 1j * k0, 0) ) * exp(-1j * k0 * z)

epscf = { "air": eps_r_air, "copper": eps_r_copper, "pml": eps_r_air }
mucf = { "air": mu_r_air, "copper": mu_r_copper, "pml": mu_r_air }

epsilon = mesh.MaterialCF(epscf, default=epscf["air"])
mu = mesh.MaterialCF(mucf, default=mucf["air"])

a = BilinearForm(fes, symmetric=True)
a += (1/mu) * curl(E_scat) * curl(v) * dx
a += -k0**2 * epsilon * E_scat * v * dx

f = LinearForm(fes)
f += (k0**2 * (epsilon - eps_r_air)) * E_inc * v * dx("copper")
f += - (1/mu_r_air) * E_inc_curl * curl(v) * dx("air")
f += - (-k0**2 * eps_r_air) * E_inc * v * dx("air")
f += - (1/mu_r_air) * E_inc_curl * curl(v) * dx("pml")
f += - (-k0**2 * eps_r_air) * E_inc * v * dx("pml")

gfu_scat = GridFunction(fes, "E_scat")

with TaskManager():
    print("Assembling matrices...")
    a.Assemble()
    f.Assemble()
    print("Solving linear system using an iterative solver...")
    preconditioner = a.mat.CreateSmoother(freedofs=fes.FreeDofs())
    
    solver = CGSolver(mat=a.mat, pre=preconditioner, printrates=True, maxsteps=2000)
    
    gfu_scat.vec.data = solver * f.vec
    
    print("Solution complete.")

gfu_total = GridFunction(fes, "E_total")
gfu_total.Set(E_inc + gfu_scat)

E_mag = Norm(gfu_total)
Draw(E_mag, mesh, "test", material="copper",clipping=True)
Draw(gfu_total)


Assembling matrices...
Solving linear system using an iterative solver...
Solution complete.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene